In [ ]:
%pip install pytube
%pip install opencv-python
%pip install -r requirements.txt

In [1]:
from pytube import YouTube
import os
import glob
import cv2


file_dir = os.getcwd()
txt_list = glob.glob(os.path.join(file_dir, 'list.txt'))

In [ ]:
for txt in txt_list:
    file_name = os.path.basename(txt)
    cls = file_name.split(".")[0]
    
    save_dir = os.path.join(file_dir,"src")
    
    if os.path.exists(save_dir) == False:
        os.mkdir(save_dir)

    print(f"Downloading videos is started")
    
    with open(txt, "r") as f:
        urllines = f.readlines()

        for idx ,url in enumerate(urllines):
            yt = YouTube(url)
            try:
                print(f"Video {yt.title} is downloading...")
                stream = yt.streams.filter(only_video=True).order_by('resolution').desc().first() #가장 화질 좋도록
                stream.download(save_dir)
            except:
                print(">>> Something is wrong, stop downloading current file...\n")
            
            
        print(f"< Downloading videos is finished >\n")

print("Done!")

In [ ]:
def extract_frame(file_dir ,file_name):
    
    filepath = os.path.join(file_dir,"src", file_name)
    video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함
    
    if not video.isOpened():
        print("Could not Open :", filepath)
        exit(0)


    length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    print("length :", length)
    print("width :", width)
    print("height :", height)
    print("fps :", fps)
    
    frame_dir = os.path.join(file_dir,"frame")
    frame_path = os.path.join(frame_dir, file_name[:-4])
    
    try:
        if not os.path.exists(frame_path):
            os.makedirs(frame_path)
            
    except OSError:
        print ('Error: Creating directory. ' +  frame_path)

    count = 0
    success, image = video.read()
    print("Read a new frame : ", success)
        
    while success:
            print('Saved frame number :', count)
            cv2.imwrite(frame_path + "/frame%d.jpg" % count, image)
            video.set(cv2.CAP_PROP_POS_MSEC,(count*1000)) #1초 마다 추출
            success, image = video.read()
            print("Read a new frame : ", success)
            count += 1

    video.release()

In [ ]:
filepath = os.path.join(file_dir, "src")
file_list = []

for file in os.listdir(filepath):
    if file.endswith((".webm",".mp4")):
        file_list.append(file)

for file in file_list:
    extract_frame(file_dir, file)

In [ ]:
import torch
import os
from shutil import copyfile,rmtree

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, custom
cwd = os.getcwd()
path = os.path.join(cwd, "frame")
dirs = os.listdir(path)

try:
    if os.path.exists(os.path.join(cwd,"runs")):
        yn = input("You've just done before. Do you want to remove 'runs' directory?\n(if not deleted, it will be reflected in the next result)\n(y/n)[n]: ")
        if yn.lower() == 'y':
            rmtree(os.path.join(cwd,"runs"))
            print("runs directory is just deleted!")
        else:
            print("it will be remained")
except OSError:
    print("Error: creating or removing directory")


for d in dirs:
        imgs = []

        # Images
        for file in os.listdir(os.path.join(path,d)):
            imgs.append(os.path.join(path,d, file))

        # Images
        for img in imgs:

        # Inference
                results = model(img)

        # Results
                crops = results.crop(save=True)  # or .show(), .save(), .print(), .pandas(), etc.

#copy croped image to summary directory

summary = os.path.join(cwd, "croped_images")

try:
    if not os.path.exists(summary):
        os.makedirs(summary)
    else:
        yn = input("Previous result directory exists. Do you want to remove? (y/n)[n]: ")
        if yn.lower() == 'y':
            rmtree(summary)
            os.makedirs(summary)
            print("you've just deleted previous result!")
        else:
            print("it wiil be remained")

except OSError:
    print("Error: Creating directory. " + summary)

from_dirs = os.path.join(cwd,"runs","detect")

idx = 0

for directory in os.listdir(from_dirs):
    car_crop_dir = os.path.join(from_dirs, directory, "crops","car")

    if not os.path.exists(car_crop_dir):
        continue

    for name in os.listdir(car_crop_dir):
        print("copying "+os.path.join(car_crop_dir, name)+"...")
        copyfile(os.path.join(car_crop_dir, name), os.path.join(summary,str(idx).zfill(7))+".jpg")
        idx+=1
print("done!")


In [ ]:
import numpy as np
import cv2
from shutil import copyfile,rmtree

cwd = os.getcwd()
path = os.path.join(cwd, "croped_images")
dest_path = os.path.join(cwd,'filtered')
files = os.listdir(path)

print("Copying all images with a minimum resolution of 300x300 or higher")

try: 
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    else:
        yn = input("You've just done before. Do you want to remove 'filtered' directory?\n(if not deleted, it will be reflected in the next result)\n(y/n)[n]: ")
        if yn.lower() == 'y':
            rmtree(dest_path)
            os.makedirs(dest_path)
            print("filtered directory is just deleted!")
        else:
            print("it will be remained")
except OSError:
    print("Error: creating or removing directory")

    
for f in files:
    file_path = os.path.join(path, f)
    dest_file = os.path.join(dest_path, f)
    
   

    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    h, w, c = img.shape
    
    if h >= 300 and w >= 300:
        print("copying "+file_path+"...")
        copyfile(file_path, dest_file)
        


In [6]:
import random

cwd = os.getcwd()
path = os.path.join(cwd,'filtered')
files = os.listdir(path)
dest_path = os.path.join(cwd,'sampled')

sampled_img = random.sample(files,5000)

try: 
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    else:
        yn = input("You've just done before. Do you want to remove 'sampled' directory?\n(if not deleted, it will be reflected in the next result)\n(y/n)[n]: ")
        if yn.lower() == 'y':
            rmtree(dest_path)
            os.makedirs(dest_path)
            print("sampled directory is just deleted!")
        else:
            print("it will be remained")
except OSError:
    print("Error: creating or removing directory")
    
for img in sampled_img:
    file_path = os.path.join(path, img)
    dest_file = os.path.join(dest_path, img)
    print("copying "+file_path+"...")
    copyfile(file_path, dest_file)

sampled directory is just deleted!
copying e:\youtube2image\filtered\0041496.jpg...
copying e:\youtube2image\filtered\0038632.jpg...
copying e:\youtube2image\filtered\0012511.jpg...
copying e:\youtube2image\filtered\0010314.jpg...
copying e:\youtube2image\filtered\0009484.jpg...
copying e:\youtube2image\filtered\0044072.jpg...
copying e:\youtube2image\filtered\0042734.jpg...
copying e:\youtube2image\filtered\0033599.jpg...
copying e:\youtube2image\filtered\0010370.jpg...
copying e:\youtube2image\filtered\0042960.jpg...
copying e:\youtube2image\filtered\0035037.jpg...
copying e:\youtube2image\filtered\0001854.jpg...
copying e:\youtube2image\filtered\0038956.jpg...
copying e:\youtube2image\filtered\0022617.jpg...
copying e:\youtube2image\filtered\0044565.jpg...
copying e:\youtube2image\filtered\0044445.jpg...
copying e:\youtube2image\filtered\0020130.jpg...
copying e:\youtube2image\filtered\0019758.jpg...
copying e:\youtube2image\filtered\0045889.jpg...
copying e:\youtube2image\filtered\